# Vector Stores and Retrievers
- LangChain's vector store and retriever abstraction.

- Integrates data retrieval (from vector database and other sources) to LLM Workflows.

- Important for RAGs 

In [7]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

text = ""
with open("speech.txt") as f:
    text = f.read()

splitter = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap = 100)
docs = splitter.create_documents([text])


In [11]:
# setting up environment Variables and llm
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

from langchain_groq import ChatGroq
llm = ChatGroq(model = "Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000251143C61B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002511386F080>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

d:\HCL Tech\Udemy\Gen AI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [14]:
# chroma is a vector store 
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(docs, embedding)

### Retrievers
- LangChain VectorStore objects do not subclass Runnables. Thus cannot be integrated directly into LangChain Expressioin Language (LCEL) chains.

- LangChain Retrievers are Runnables. They are designed to be integrated with LCEL.

- `synchronous invoke`, `asynchronous invoke` and  `batch operations`

In [ ]:
# vectorstore.as_retriever()
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

# we will try to create Runnable by 2 ways
# 1. Runnable Lambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["world", "coffee"])

# the RunnableLambda is basically using the paramenters we give in batch([]) to run the similarity_search

[[Document(id='321c943b-515f-426d-91b0-5f8a0f86807e', metadata={}, page_content='In a world where digital currents flow like rivers through silicon valleys, the convergence of ideas gives birth to innovations beyond imagination. The humming of servers replaces the songs of birds, and every keystroke echoes with the possibility of change. Amidst this backdrop, a quiet revolution brews—fueled not by steel and fire, but by code and thought. Language models whisper insights, data')],
 [Document(id='59e0f09d-653d-4758-b4a9-a744934b842c', metadata={}, page_content='brews—fueled not by steel and fire, but by code and thought. Language models whisper insights, data dances in patterns unseen, and human curiosity dares to ask questions machines strive to answer.')]]

In [16]:
# a better technique is as_retriever
# creates a VectorStoreRetriever

retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":1}
)
retriever.batch(["world", "paper"])

[[Document(id='321c943b-515f-426d-91b0-5f8a0f86807e', metadata={}, page_content='In a world where digital currents flow like rivers through silicon valleys, the convergence of ideas gives birth to innovations beyond imagination. The humming of servers replaces the songs of birds, and every keystroke echoes with the possibility of change. Amidst this backdrop, a quiet revolution brews—fueled not by steel and fire, but by code and thought. Language models whisper insights, data')],
 [Document(id='8ff345e2-0478-40c0-93a5-32100c5b6ce7', metadata={}, page_content='In a coffee shop, a startup founder sketches out an architecture on a napkin, unsure whether it will disrupt an industry or crash on launch. Down the street, a graphic designer experiments with generative art, watching in awe as a diffusion model paints what she only dreamed. New languages emerge—not of grammar, but of instructions, tokens, and layers. New literacies are demanded not by')]]

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_messages([
    ("human", "Answer the questions based on provided context only: \n{question} \n\nContext: \n{context}")
])

# runnablepassthrough means we'll be getting thast specific value in the invoke method
rag_chain = {"context":retriever, "question":RunnablePassthrough()} | prompt | llm 

response = rag_chain.invoke("Tell me about the world")

In [21]:
response.content

'Based on the provided context, here are my answers:\n\nWhat is the world like?\n\nThe world is a digital one, where digital currents flow like rivers through silicon valleys, replacing the natural sounds of birdsong.\n\nWhat is happening in this world?\n\nA quiet revolution is brewing, fueled by code and thought, rather than traditional means of revolution like steel and fire.\n\nWhat are the key features of this world?\n\nThe key features of this world are the convergence of ideas, innovations beyond imagination, and the role of language models and data in driving change.'